In [1]:
import pandas as pd
import sqlite3

# Chemins des fichiers CSV
chemin_csv_recensement = r'D:\bureau\BD&AI 1\s3\BD\with python\projet\ChicagoCensusData.csv'
chemin_csv_ecoles = r'D:\bureau\BD&AI 1\s3\BD\with python\projet\ChicagoPublicSchools.csv'
chemin_csv_criminalite = r'D:\bureau\BD&AI 1\s3\BD\with python\projet\ChicagoCrimeData.csv'

In [2]:
# Chargement des données CSV
df_recensement = pd.read_csv(chemin_csv_recensement, delimiter=',')
df_ecoles = pd.read_csv(chemin_csv_ecoles, delimiter=',')
df_criminalite = pd.read_csv(chemin_csv_criminalite, delimiter=',')

In [3]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('FinalDB.db')

In [4]:
# Suppression des tables si elles existent
conn.execute('DROP TABLE IF EXISTS table_recensement')
conn.execute('DROP TABLE IF EXISTS table_ecoles')
conn.execute('DROP TABLE IF EXISTS table_criminalite')

In [5]:
# Enregistrement des DataFrames dans des tables SQLite
df_recensement.to_sql('table_recensement', conn)
df_ecoles.to_sql('table_ecoles', conn)
df_criminalite.to_sql('table_criminalite', conn)

533

In [6]:
# Exécution des requêtes SQL
requete_nombre_criminalite = 'SELECT COUNT(ID) FROM table_criminalite'
resultat_nombre_criminalite = pd.read_sql(requete_nombre_criminalite, conn)
print(resultat_nombre_criminalite)

   COUNT(ID)
0        533


In [7]:
requete_zones_faible_revenu = 'SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME FROM table_recensement WHERE PER_CAPITA_INCOME < 11000'
resultat_zones_faible_revenu = pd.read_sql(requete_zones_faible_revenu, conn)
print(resultat_zones_faible_revenu)

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME
0                   26.0  West Garfield Park
1                   30.0      South Lawndale
2                   37.0         Fuller Park
3                   54.0           Riverdale


In [8]:
requete_criminalite_recente = "SELECT CASE_NUMBER, DATE FROM table_criminalite WHERE (strftime('%Y', 'now') - strftime('%Y', DATE)) < 18"
resultat_criminalite_recente = pd.read_sql(requete_criminalite_recente, conn)
print(resultat_criminalite_recente)

    CASE_NUMBER        DATE
0      HT233595  2011-04-04
1      HT133522  2010-12-30
2      HZ138551  2016-02-02
3      HS540106  2010-09-29
4      HZ534771  2016-11-30
..          ...         ...
306    HZ378355  2016-08-02
307    HX258445  2014-05-12
308    HZ192829  2016-03-01
309    HZ133234  2016-01-29
310    JA143710  2017-02-06

[311 rows x 2 columns]


In [9]:
requete_info_table_criminalite = "PRAGMA table_info(table_criminalite)"
resultat_info_table_criminalite = pd.read_sql(requete_info_table_criminalite, conn)
print(resultat_info_table_criminalite)

    cid                   name     type  notnull dflt_value  pk
0     0                  index  INTEGER        0       None   0
1     1                     ID  INTEGER        0       None   0
2     2            CASE_NUMBER     TEXT        0       None   0
3     3                   DATE     TEXT        0       None   0
4     4                  BLOCK     TEXT        0       None   0
5     5                   IUCR     TEXT        0       None   0
6     6           PRIMARY_TYPE     TEXT        0       None   0
7     7            DESCRIPTION     TEXT        0       None   0
8     8   LOCATION_DESCRIPTION     TEXT        0       None   0
9     9                 ARREST  INTEGER        0       None   0
10   10               DOMESTIC  INTEGER        0       None   0
11   11                   BEAT  INTEGER        0       None   0
12   12               DISTRICT  INTEGER        0       None   0
13   13                   WARD     REAL        0       None   0
14   14  COMMUNITY_AREA_NUMBER     REAL 

In [10]:
requete_kidnapping_recent = "SELECT * FROM table_criminalite WHERE PRIMARY_TYPE = 'KIDNAPPING' AND (strftime('%Y', 'now') - strftime('%Y', DATE)) < 18"
resultat_kidnapping_recent = pd.read_sql(requete_kidnapping_recent, conn)
print(resultat_kidnapping_recent)

   index       ID CASE_NUMBER        DATE                 BLOCK  IUCR  \
0    520  5276766    HN144152  2007-01-26  050XX W VAN BUREN ST  1792   

  PRIMARY_TYPE               DESCRIPTION LOCATION_DESCRIPTION  ARREST  ...  \
0   KIDNAPPING  CHILD ABDUCTION/STRANGER               STREET       0  ...   

   DISTRICT  WARD  COMMUNITY_AREA_NUMBER  FBICODE  X_COORDINATE Y_COORDINATE  \
0        15  29.0                   25.0       20     1143050.0    1897546.0   

   YEAR   LATITUDE  LONGITUDE                       LOCATION  
0  2007  41.874908 -87.750249  (41.874908413, -87.750249307)  

[1 rows x 22 columns]


In [11]:
requete_relation_criminalite_ecole = "SELECT t.PRIMARY_TYPE, x.NAME_OF_SCHOOL FROM table_criminalite t JOIN table_ecoles x ON t.LOCATION = x.LOCATION"
resultat_relation_criminalite_ecole = pd.read_sql(requete_relation_criminalite_ecole, conn)
print(resultat_relation_criminalite_ecole)

Empty DataFrame
Columns: [PRIMARY_TYPE, NAME_OF_SCHOOL]
Index: []


In [12]:
requete_moyenne_score_securite = "SELECT `Elementary, Middle, or High School`, AVG(SAFETY_SCORE) AS avg_safety_score FROM table_ecoles WHERE `Elementary, Middle, or High School` IN ('MS', 'HS', 'ES') GROUP BY `Elementary, Middle, or High School`"
resultat_moyenne_score_securite = pd.read_sql(requete_moyenne_score_securite, conn)
print(resultat_moyenne_score_securite)

  Elementary, Middle, or High School  avg_safety_score
0                                 ES         49.520384
1                                 HS         49.623529
2                                 MS         48.000000


In [13]:
requete_pauvrete_revenu = "SELECT COMMUNITY_AREA_NUMBER, PER_CAPITA_INCOME FROM table_recensement ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5"
resultat_pauvrete_revenu = pd.read_sql(requete_pauvrete_revenu, conn)
print(resultat_pauvrete_revenu)

   COMMUNITY_AREA_NUMBER  PER_CAPITA_INCOME
0                   54.0               8201
1                   37.0              10432
2                   68.0              11888
3                   29.0              12034
4                   27.0              12961


In [14]:
requete_zone_criminalite_max = "SELECT v.COMMUNITY_AREA_NUMBER, v.COMMUNITY_AREA_NAME, COUNT(b.COMMUNITY_AREA_NUMBER) as crime_count FROM table_recensement v LEFT JOIN table_criminalite b ON v.COMMUNITY_AREA_NUMBER = b.COMMUNITY_AREA_NUMBER GROUP BY v.COMMUNITY_AREA_NUMBER, v.COMMUNITY_AREA_NAME ORDER BY crime_count DESC LIMIT 1"
resultat_zone_criminalite_max = pd.read_sql(requete_zone_criminalite_max, conn)
print(resultat_zone_criminalite_max)

   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  crime_count
0                   25.0              Austin           43


In [15]:
requete_zone_hardship_max = "SELECT COMMUNITY_AREA_NAME FROM table_recensement WHERE HARDSHIP_INDEX = (SELECT MAX(HARDSHIP_INDEX) FROM table_recensement)"
resultat_zone_hardship_max = pd.read_sql(requete_zone_hardship_max, conn)
print(resultat_zone_hardship_max)

  COMMUNITY_AREA_NAME
0           Riverdale


In [16]:
requete_zone_criminalite_max_comm = "SELECT COMMUNITY_AREA_NAME FROM table_recensement WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM table_criminalite GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(*) DESC LIMIT 1)"
resultat_zone_criminalite_max_comm = pd.read_sql(requete_zone_criminalite_max_comm, conn)
print(resultat_zone_criminalite_max_comm)

  COMMUNITY_AREA_NAME
0              Austin
